In [71]:
import pandas as pd
import numpy as np
import os

In [7]:
files = [i for i in os.listdir('.') if i.endswith('xlsx')]

In [61]:
def replaceColumnLabel(df, row = 0, fillna = ''):
  col = df.iloc[row].fillna(fillna)
  df = df.iloc[row+1:]
  df.columns = col
  return df

In [86]:
def process(path):
  year = re.findall('\d+', path)
  df = pd.read_excel(path, skiprows=1)
  international = df.loc[:,'Keberangkatan pada Penerbangan Internasional':]
  international = replaceColumnLabel(international, row=1, fillna='airport').dropna()
  international = np.ravel(international)
  international = np.delete(international, np.where(international == '-'))
  international_sum = np.sum(international.astype(int))
  national = df.loc[:, 'Keberangkatan pada Penerbangan Domestik':'Keberangkatan pada Penerbangan Internasional'].drop('Keberangkatan pada Penerbangan Internasional', axis=1)
  national = replaceColumnLabel(national, row=1, fillna='airport').dropna()
  national = np.ravel(national)
  national = np.delete(national, np.where(national == '-'))
  national_sum = np.sum(national.astype(int))
  res_df = pd.DataFrame({'tahun': year, 'nasional': national_sum, 'internasional':international_sum})
  return res_df

In [89]:
dfs = [process(i) for i in files]

In [92]:
df = pd.concat(dfs).sort_values('tahun')
df

,tahun,nasional,internasional
0,2019,36494088,17108874
0,2020,32511290,6520358
0,2021,8759617,356016


In [93]:
df.to_csv('total keberangkatan melalui airport 2019-2020.csv', index=False)